In [1]:
#-- Set up configurations / parameters
retrain = False # retrain previously existing model
ndown = 4 # number of 'down' steps
ninit = 32 #number of channels to start with
dropout_frac = 0.2 # dropout fraction
n_batch = 15
n_epochs = 30
n_test = 500
ratio = 727 # penalization ratio for GL and non-GL points based on smaller dataaset
mod_lbl = 'atrous' #'unet'
if mod_lbl == 'unet':
  mod_str = '{0}_{1}init_{2}down_drop{3:.1f}_customLossR{4}'.format(mod_lbl,ninit,ndown,
                                                        dropout_frac,ratio)
elif mod_lbl == 'atrous':
  mod_str = '{0}_{1}init_drop{2:.1f}_customLossR{3}'.format(mod_lbl,ninit,dropout_frac,ratio)
else:
  print('model label not matching.')
print(mod_str)

atrous_32init_drop0.2_customLossR727


In [2]:
!pip install rasterio

     |████████████████████████████████| 18.2MB 1.3MB/s 


In [3]:
#-- Import Modules
import os
import sys
import imp
import random
from osgeo import gdal,osr
import rasterio
import numpy as np
import matplotlib.pyplot as plt 
from google.colab import drive
import keras
from keras import backend as K
from keras.preprocessing import image

Using TensorFlow backend.


In [4]:
#-- Mount Google Drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
#-- Directory setup
colabdir = '/content/gdrive/My Drive/Colab Notebooks/'
gl_dir = '/content/gdrive/My Drive/GL_Learning/'
ddir = '/content/gdrive/Shared drives/GROUNDING_LINE_TEAM_DRIVE/ML_Yara/S1_Pope-Smith-Kohler/UNUSED/coco_PSK-UNUSED_with_null/'
output_dir = '/content/gdrive/Shared drives/GROUNDING_LINE_TEAM_DRIVE/ML_Yara/S1_Pope-Smith-Kohler/UNUSED/pred_PSK-UNUSED_with_null/'

Try unzipping file on google drive directly.

In [ ]:
!unzip /content/gdrive/Shared\ drives/GROUNDING_LINE_TEAM_DRIVE/ML_Yara/S1_Pope-Smith-Kohler/UNUSED/coco_PSK-UNUSED_with_null.zip -d /content/gdrive/Shared\ drives/GROUNDING_LINE_TEAM_DRIVE/ML_Yara/S1_Pope-Smith-Kohler/UNUSED/coco_PSK-UNUSED_with_null

Streaming output truncated to the last 5000 lines.
  inflating: /content/gdrive/Shared drives/GROUNDING_LINE_TEAM_DRIVE/ML_Yara/S1_Pope-Smith-Kohler/UNUSED/coco_PSK-UNUSED_with_null/coco_gl_010_180418-180424-180430-180506_010536-021607-010711-021782_T095931_T095932_x1154_y0512_DIR01.tif  
  inflating: /content/gdrive/Shared drives/GROUNDING_LINE_TEAM_DRIVE/ML_Yara/S1_Pope-Smith-Kohler/UNUSED/coco_PSK-UNUSED_with_null/coco_gl_010_180418-180424-180430-180506_010536-021607-010711-021782_T095931_T095932_x1154_y0570_DIR00.tif  
  inflating: /content/gdrive/Shared drives/GROUNDING_LINE_TEAM_DRIVE/ML_Yara/S1_Pope-Smith-Kohler/UNUSED/coco_PSK-UNUSED_with_null/coco_gl_010_180418-180424-180430-180506_010536-021607-010711-021782_T095931_T095932_x1154_y1024_DIR01.tif  
  inflating: /content/gdrive/Shared drives/GROUNDING_LINE_TEAM_DRIVE/ML_Yara/S1_Pope-Smith-Kohler/UNUSED/coco_PSK-UNUSED_with_null/coco_gl_010_180418-180424-180430-180506_010536-021607-010711-021782_T095931_T095932_x1154_y1082_DIR00

In [6]:
#-- Get list of images
fileList = os.listdir(ddir)
file_list = [f for f in fileList if (f.endswith('.tif') and f.startswith('coco'))]
N = len(file_list)
N

5301

In [7]:
#-- read first file to get dimensions
raster = rasterio.open(os.path.join(ddir,file_list[0]))
h = raster.height
wi = raster.width
ch = raster.count

In [8]:
print(h,wi,ch)

512 512 1


In [9]:
#-- set channel to 2 because there are actually real and imaginary components
ch = 2

In [10]:
#-- Import model
mod_module = imp.load_source('nn_model',os.path.join(colabdir,'nn_model.py'))
#-- set up model
if mod_lbl == 'unet':
  print('loading unet model')
  model = mod_module.unet_model_double_dropout(height=h,width=wi,channels=ch, 
                                        n_init=ninit,n_layers=ndown,
                                        drop=dropout_frac)
elif mod_lbl == 'atrous':
  print("loading atrous model")
  model = mod_module.nn_model_atrous_double_dropout(height=h,width=wi,
                                                channels=ch,
                                                n_filts=ninit,
                                                drop=dropout_frac)
else:
  print('Model label not correct.')

loading atrous model


In [11]:
#-- define custom loss function
def customLoss(yTrue,yPred):
  return -1*K.mean(ratio*(yTrue*K.log(yPred+1e-32)) + ((1. - yTrue)*K.log(1-yPred+1e-32)))

In [12]:
#-- compile imported model
model.compile(loss=customLoss,optimizer='adam',
              metrics=['accuracy'])

In [13]:
#-- checkpoint file
chk_file = os.path.join(gl_dir,'{0}_weights.h5'.format(mod_str))
print(chk_file)
#-- if file exists, read model from file
if os.path.isfile(chk_file):
  print('Check point exists; loading model from file.')
  #-- load weights
  model.load_weights(chk_file)
else:
  print('Model does not previously exist.')

/content/gdrive/My Drive/GL_Learning/atrous_32init_drop0.2_customLossR727_weights.h5
Check point exists; loading model from file.


In [14]:
#-------------------------------
#-- Run model on data
#-------------------------------
#-- make output directory
out_dir = os.path.join(output_dir,'{0}.dir'.format(mod_str))
if (not os.path.isdir(out_dir)):
	os.mkdir(out_dir)
#-- read 500 files at a time (memory bottleneck)
cc = 0
while (cc < N):
	print(cc)
	#-- Read data all at once
	imgs = np.ones((500,h,wi,ch))
	trans = [None]*500
	for i,f in enumerate(file_list[cc:cc+500]):
		#-- read image
		raster = rasterio.open(os.path.join(ddir,f))
		try:
			imgs[i,:,:,0] = raster.read(1).real
			imgs[i,:,:,1] = raster.read(1).imag
		except:
			print('Skipping %s'%f)
			imgs[i,:,:,0] = None
			imgs[i,:,:,1] = None
		#-- get transformation matrix
		trans[i] = raster.transform
		raster.close()

	out_crs = raster.crs.to_epsg()
	out_imgs = model.predict(imgs, batch_size=1, verbose=1)
	out_imgs = out_imgs.reshape(out_imgs.shape[0],h,wi,out_imgs.shape[2])
	
	#-- save output images
	for i,f in enumerate(file_list[cc:cc+500]):
		#-- get pixel size
		x1,y1 = rasterio.transform.xy(trans[i], 0, 0)
		x2,y2 = rasterio.transform.xy(trans[i], 0, 1)
		x3,y3 = rasterio.transform.xy(trans[i], 1, 0)
		dx = np.abs(x2 - x1)
		dy = np.abs(y3 - y1)
		#-- Now find the coordinates of the upper left corner of scene based on total size
		#-- note the x1,y1 refers to position list_x0[i],list_y0[i]
		list_x0=int(f.split('_x')[1].split('_y')[0])
		list_y0=int(f.split('_y')[1].split('_DIR')[0])
		x_orig = x1 - (dx*list_x0)
		y_orig = y1 + (dy*list_y0)

		#-- get transformation for output
		#-- output as geotiff
		driver = gdal.GetDriverByName("GTiff")
		#-- set up the dataset with compression options (1 is for band 1)
		OPTS = ['COMPRESS=LZW']
		ds = driver.Create(os.path.join(out_dir,os.path.basename(f).replace('coco','pred')), \
			h, wi, 1, gdal.GDT_Float32, OPTS)
		#-- top left x, w-e pixel resolution, rotation
		#-- top left y, rotation, n-s pixel resolution
		ds.SetGeoTransform([x_orig, dx, 0, y_orig, 0, -dy])
		#-- set the reference info
		srs = osr.SpatialReference()
		srs.ImportFromEPSG(out_crs)
		#-- export
		ds.SetProjection( srs.ExportToWkt() )
		#-- write to geotiff array
		ds.GetRasterBand(1).WriteArray(out_imgs[i].reshape(h,wi))
		ds.FlushCache()
		ds = None
		#-- also save as image
		im = image.array_to_img(out_imgs[i]) 
		im.save(os.path.join(out_dir,os.path.basename(f).replace('coco','pred').replace('tif','png')))
	#-- increment counter
	cc += 500

0
500/500 [==============================] - 32s 63ms/step
500
Skipping coco_gl_010_180927-181003-181003-181009_023882-012986-012986-024057_T100022_T095940_x1536_y2048_DIR11.tif
500/500 [==============================] - 25s 49ms/step
1000
500/500 [==============================] - 25s 50ms/step
1500
500/500 [==============================] - 25s 50ms/step
2000
500/500 [==============================] - 25s 50ms/step
2500
500/500 [==============================] - 27s 54ms/step
3000
500/500 [==============================] - 25s 50ms/step
3500
500/500 [==============================] - 27s 53ms/step
4000
500/500 [==============================] - 25s 51ms/step
4500
500/500 [==============================] - 28s 56ms/step
5000
500/500 [==============================] - 27s 54ms/step


In [ ]:
#-------------------------------
#-- Run model on data
#-------------------------------
num = 800
#-- make output directory
out_dir = os.path.join(output_dir,'{0}.dir'.format(mod_str))
if (not os.path.isdir(out_dir)):
    os.mkdir(out_dir)
while (cc < N):
    #-- read "num" files at a time
    print(cc)
    #-- Read data all at once
    imgs = np.ones((num,h,wi,ch))
    trans = [None]*num
    for i,f in enumerate(file_list[cc:cc+num]):
        #-- read image
        raster = rasterio.open(os.path.join(ddir,f))
        try:
            imgs[i,:,:,0] = raster.read(1).real
            imgs[i,:,:,1] = raster.read(1).imag
        except:
            print('Skipping %s'%f)
            imgs[i,:,:,0] = None
            imgs[i,:,:,1] = None
        #-- get transformation matrix
        trans[i] = raster.transform
        raster.close()

    out_crs = raster.crs.to_epsg()
    out_imgs = model.predict(imgs, batch_size=1, verbose=1)
    out_imgs = out_imgs.reshape(out_imgs.shape[0],h,wi,out_imgs.shape[2])

    #-- save output images
    for i,f in enumerate(file_list[cc:cc+num]):
        #-- get pixel size
        x_orig,y_orig = rasterio.transform.xy(trans[i], 0, 0)
        x2,y2 = rasterio.transform.xy(trans[i], 0, 1)
        x3,y3 = rasterio.transform.xy(trans[i], 1, 0)
        dx = np.abs(x2 - x_orig)
        dy = np.abs(y3 - y_orig)

        #-- get transformation for output
        #-- output as geotiff
        driver = gdal.GetDriverByName("GTiff")
        #-- set up the dataset with compression options (1 is for band 1)
        OPTS = ['COMPRESS=LZW']
        ds = driver.Create(os.path.join(out_dir,os.path.basename(f).replace('coco','pred')),\
            h, wi, 1, gdal.GDT_Float32, OPTS)
        #-- top left x, w-e pixel resolution, rotation
        #-- top left y, rotation, n-s pixel resolution
        ds.SetGeoTransform([x_orig, dx, 0, y_orig, 0, -dy])
        #-- set the reference info
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(out_crs)
        #-- export
        ds.SetProjection( srs.ExportToWkt() )
        #-- write to geotiff array
        ds.GetRasterBand(1).WriteArray(out_imgs[i].reshape(h,wi))
        ds.FlushCache()
        ds = None
        #-- also save as image
#         im = image.array_to_img(out_imgs[i]) 
#         im.save(os.path.join(out_dir,os.path.basename(f).replace('coco','pred').replace('tif','png')))
    #-- increment counter
    cc += num